In [11]:
from datasets import load_dataset

import torch
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
from pprint import pprint
from bracketology_rl import Bracket
from bracketgym1game import TournamentEnv

In [12]:
from stable_baselines3 import PPO,A2C,SAC,DQN,DDPG
from sb3_contrib import RecurrentPPO,ARS,QRDQN
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.env_util import make_vec_env
import torch
from torch.utils.data import DataLoader, Dataset


In [13]:
from imitation.data.rollout import TrajectoryAccumulator
from imitation.data import serialize
from imitation.data.rollout import rollout_stats

In [14]:
years = list(range(2003,2019))
year=years[1]

bracket = Bracket(2019)
# bracket.result

In [15]:



team_data=pd.read_csv('Process_data/pm_w_names.csv')
env=TournamentEnv(team_data=team_data,weight_scores=True,discrete=True,loading_bar=True)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [16]:
done=False
i=0
env.reset()
while not done:
    i+=1
    act=env.get_correct_action()

    if (i%10)==0:
        act=int(np.random.choice([0,1]))
    print(i,act,env.current_round)
    next_obs, reward, done,truncated, info=env.step(action=act)

        

1 0 1
2 0 1
3 0 1
4 0 1
5 0 1
6 0 1
7 0 1
8 0 1
9 0 1
10 0 1
11 0 1
12 0 1
13 0 1
14 0 1
15 0 1
16 0 1
17 0 1
18 0 1
19 0 1
20 1 1
21 0 1
22 0 1
23 0 1
24 0 1
25 0 1
26 0 1
27 0 1
28 0 1
29 0 1
30 1 1
31 0 1
32 0 1
33 0 2
34 1 2
35 0 2
36 1 2
37 0 2
38 1 2
39 0 2
40 0 2
41 0 2
42 0 2
43 1 2
44 0 2
45 0 2
['Wisconsin', 'Kansas St']
['Duke', 'Gonzaga', 'North Carolina', 'Virginia', 'Kentucky', 'Michigan', 'Michigan St', 'Tennessee', 'Houston', 'LSU', 'Purdue', 'Texas Tech', 'Florida St', 'Kansas', 'Virginia Tech', 'Auburn', 'Buffalo', 'Maryland', 'Villanova', 'Wofford', 'Baylor', 'Oklahoma', 'UCF', 'Washington', 'Florida', 'Iowa', 'Minnesota', 'Ohio St', 'Liberty', 'Murray St', 'Oregon', 'UC Irvine']
32
46 ['oops'] 2


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
env.result_dict

In [ ]:
num_dems=500
traj_acum=TrajectoryAccumulator()
demonstrations=[]
j=0
for i in range(num_dems):
    obs,info=env.reset()

    terminated=False
    step={'obs':obs}
    traj_acum.add_step(key=i,step_dict=step)
    while not terminated:
        act=env.get_correct_action()
        if (j%40)==0:
            act=np.random.choice([0,1])
        next_obs, reward, terminated,truncated, info=env.step(action=act,)
        
        step={'obs':obs,
            'acts':act,
            'rews':np.array(reward),
            'infos':info}
        traj_acum.add_step(key=i,
                        step_dict=step,
                        )
        # print(reward,train_env.trade_week_start,train_env.current_time)
        obs=next_obs
    
    traj=traj_acum.finish_trajectory(key=i,terminal=terminated)
    demonstrations.append(traj)

In [ ]:
stats = rollout_stats(demonstrations)
print(stats["return_mean"])

In [ ]:
@dataclass
class DecisionTransformerGymDataCollator:
    return_tensors: str = "pt"
    max_len: int = 20 #subsets of the episode we use for training
    state_dim: int = 17  # size of state space
    act_dim: int = 6  # size of action space
    max_ep_len: int = 63 # max episode length in the dataset
    scale: float = 1000.0  # normalization of rewards/returns
    state_mean: np.array = None  # to store state means
    state_std: np.array = None  # to store state stds
    p_sample: np.array = None  # a distribution to take account trajectory lengths
    n_traj: int = 0 # to store the number of trajectories in the dataset

    def __init__(self, dataset) -> None:
        self.act_dim = len(dataset[0]["actions"][0])
        self.state_dim = len(dataset[0]["observations"][0])
        self.dataset = dataset
        # calculate dataset stats for normalization of states
        states = []
        traj_lens = []
        for obs in dataset["observations"]:
            states.extend(obs)
            traj_lens.append(len(obs))
        self.n_traj = len(traj_lens)
        states = np.vstack(states)
        self.state_mean, self.state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
        
        traj_lens = np.array(traj_lens)
        self.p_sample = traj_lens / sum(traj_lens)

    def _discount_cumsum(self, x, gamma):
        discount_cumsum = np.zeros_like(x)
        discount_cumsum[-1] = x[-1]
        for t in reversed(range(x.shape[0] - 1)):
            discount_cumsum[t] = x[t] + gamma * discount_cumsum[t + 1]
        return discount_cumsum

    def __call__(self, features):
        batch_size = len(features)
        # this is a bit of a hack to be able to sample of a non-uniform distribution
        batch_inds = np.random.choice(
            np.arange(self.n_traj),
            size=batch_size,
            replace=True,
            p=self.p_sample,  # reweights so we sample according to timesteps
        )
        # a batch of dataset features
        s, a, r, d, rtg, timesteps, mask = [], [], [], [], [], [], []
        
        for ind in batch_inds:
            # for feature in features:
            feature = self.dataset[int(ind)]
            si = random.randint(0, len(feature["rewards"]) - 1)

            # get sequences from dataset
            s.append(np.array(feature["observations"][si : si + self.max_len]).reshape(1, -1, self.state_dim))
            a.append(np.array(feature["actions"][si : si + self.max_len]).reshape(1, -1, self.act_dim))
            r.append(np.array(feature["rewards"][si : si + self.max_len]).reshape(1, -1, 1))

            d.append(np.array(feature["dones"][si : si + self.max_len]).reshape(1, -1))
            timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))
            timesteps[-1][timesteps[-1] >= self.max_ep_len] = self.max_ep_len - 1  # padding cutoff
            rtg.append(
                self._discount_cumsum(np.array(feature["rewards"][si:]), gamma=1.0)[
                    : s[-1].shape[1]   # TODO check the +1 removed here
                ].reshape(1, -1, 1)
            )
            if rtg[-1].shape[1] < s[-1].shape[1]:
                print("if true")
                rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)

            # padding and state + reward normalization
            tlen = s[-1].shape[1]
            s[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, self.state_dim)), s[-1]], axis=1)
            s[-1] = (s[-1] - self.state_mean) / self.state_std
            a[-1] = np.concatenate(
                [np.ones((1, self.max_len - tlen, self.act_dim)) * -10.0, a[-1]],
                axis=1,
            )
            r[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), r[-1]], axis=1)
            d[-1] = np.concatenate([np.ones((1, self.max_len - tlen)) * 2, d[-1]], axis=1)
            rtg[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), rtg[-1]], axis=1) / self.scale
            timesteps[-1] = np.concatenate([np.zeros((1, self.max_len - tlen)), timesteps[-1]], axis=1)
            mask.append(np.concatenate([np.zeros((1, self.max_len - tlen)), np.ones((1, tlen))], axis=1))

        s = torch.from_numpy(np.concatenate(s, axis=0)).float()
        a = torch.from_numpy(np.concatenate(a, axis=0)).float()
        r = torch.from_numpy(np.concatenate(r, axis=0)).float()
        d = torch.from_numpy(np.concatenate(d, axis=0))
        rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).float()
        timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).long()
        mask = torch.from_numpy(np.concatenate(mask, axis=0)).float()

        return {
            "states": s,
            "actions": a,
            "rewards": r,
            "returns_to_go": rtg,
            "timesteps": timesteps,
            "attention_mask": mask,
        }
